# System on Chip design auxiliaries

For mass generation of bus decoders, a register bit map must be associated with corresponding control, status or data signals.
This SoC concept follows the [*MaSoCist*](https://github.com/hackfin/MaSoCisthttps://github.com/hackfin/MaSoCist) register map design rules:

* Registers are mapped into memory space and are accessed by an address, hence.
* They can be flagged read-only, write-only or volatile:
  * READONLY: Writing to the register has no effect
  * WRITEONLY: Reading from this register returns an undefined value
  * VOLATILE: Write or read access triggers a pulse on the corresponding `select` lines.
    This allows to implement `W1C` (write one to clear) behaviour, or optimized data in/out.
* Registers contain bit fields that can be READONLY or WRITEONLY
* Two register definitions (one READONLY, one WRITEONLY) can be mapped to one address. This is used for data I/O.

To skip the entire *myIRL* definition, jump to the actual [register map decoder implementation](#Register-map-decoder)

We create a Container extension that takes a register as argument:

In [1]:
import sys
sys.path.insert(0, "../../")

In [2]:
from myirl.library.registers import Register, NamedBitfield
from myirl.kernel.components import ContainerExtension, ContainerBase, Signal
from myirl.library.bulksignals import BulkWrapperSig
from myhdl import intbv
from myirl.kernel.sensitivity import hdlmacro

class BF(NamedBitfield):
    READONLY = 0x01
    WRITEONLY = 0x02
    
    def __init__(self, name, msb, lsb, flags = 0):
        super().__init__(name, msb, lsb)
        self.flags = flags

class Reg(Register):
    VOLATILE = 0x04
    READONLY = 0x01
    WRITEONLY = 0x02

    def __init__(self, size, bitfields, flags = 0):
        super().__init__(size, bitfields)
        self.flags = flags
    
    def has_select(self, flag):
        if (self.flags & Reg.VOLATILE) and not (self.flags & flag):
            return True
    
        return False

class RegAssignmentMixin:
    def set(self, other):
        yield []
    
    def assign(self, other):
        yield []

          
class RegisterSignal(ContainerExtension):
    def __init__(self, name, reg, virtual = None):
        if name is None:
            name = kernel.utils.get_uuid('reg_')
        if not isinstance(reg, Register):
            raise TypeError("expects register as argument")
            
        bitfields = reg.members().items()
            
        signals = {
            n : Signal(intbv()[i.msb + 1:i.lsb] if i.msb != i.lsb else bool(),
                       name = i.name)
            for n, i in bitfields
        }
        # Selection signals ('W1C' etc.)
        
        select_sigs = {}

        if reg.flags & reg.VOLATILE:
            if not reg.flags & reg.READONLY:
                select_sigs["sel_w"] = Signal(bool())
            if not reg.flags & reg.WRITEONLY:
                select_sigs["sel_r"] = Signal(bool())

        inputs = filter(lambda t: (t[1].flags & BF.WRITEONLY) == False, bitfields)
        outputs = filter(lambda t: (t[1].flags & BF.READONLY) == False, bitfields)
        
        input_container  = self.create(name + '_read', { n : signals[n] for n, _ in inputs }, ())
        output_container = self.create(name + '_write', { n : signals[n] for n, _ in outputs }, ())

        select_container = self.create(name + '_sel', select_sigs, ())
        
        sigs = {}
        
        if input_container is not None:
            sigs['read'] = input_container
        if output_container is not None:
            output_container.driven = True
            sigs['write'] = output_container
        if select_container is not None:
            select_container.driven = True
            sigs['select'] =  select_container

        super().__init__(name, sigs, template = reg, virtual = True, twoway = True)
        
    def create(self, sign, children, bases = (RegAssignmentMixin, )):
        "Dynamically create a subclass from self._type"
        if len(children) == 0:
            return None
        d = { '_templ' : children, '_virtual' : True,
              '_rank' : ContainerBase._rank }

        container = type(sign, (BulkWrapperSig, *bases), d)
        inst = container()
        inst._populate(children)
        return inst
                
    def alias(self, name):
        "We need a different .alias() method, as we pass a Register for a new object"
        signals = {}
        for n, s in self._members.items():
            nn = name + '_' + n
            sig = s.alias(nn)
            signals[nn] = sig
            
        new = type(self)(name, self._template)
        new.rename(name) # Need explicit rename here XXX
        return new
    
    # Here we don't need a @hdlmacro, because we reuse the Register.assign() method
    def assign(self, data):
        d = {}
        try:
            readport_members = self.get_children()['read'].members().items()
            for n, i in readport_members:
                bf = self._template.bfmap[n]
                d[n] = i
 
            gen = self._template.assign(data, **d)
            return gen
        except KeyError:
            return None

    @hdlmacro
    def select_reset(self):
        try:
            members = self.get_children()['select'].members().items()
            gen = []
            for n, i in members:
                gen.append(i.set(False))
            yield gen
        except KeyError:
            yield []
        
    @hdlmacro
    def set(self, other):
        gen = []
        try:
            w = self.get_children()['write']
            for n, i in w.members().items():
                bf = self._template.bfmap[n]
                if bf.msb == bf.lsb:
                    gen.append(i.set(other[bf.msb]))
                else:
                    gen.append(i.set(other[bf.msb + 1: bf.lsb]))

            yield gen
        except KeyError:
            yield []

## Register definitions

Add a few register with bit fields and flags:

In [3]:

reg01 = Reg(16,
    [
        BF("im", 3, 1, flags = BF.READONLY),
        BF("ex", 7, 6),
        BF("inv", 4, 4, flags = BF.WRITEONLY),
        BF("mode", 14, 10)
    ]
)

reg02 = Reg(16,
    [
        BF("gna", 6, 1),
        BF("reset", 7, 7)
    ],
    flags = Reg.VOLATILE | Reg.WRITEONLY
)



## Register decoder table generator

The dictionary below defines an address mapping to registers, containing specific bit fields:

In [4]:
regdesc = {
    0x01: ['stat', reg01],
    0x02: ['ctrl', reg02],
    0x04: ['TXD',  Reg(16, [ BF("DATA", 15, 0)], flags = Reg.WRITEONLY | Reg.VOLATILE) ],
    0x05: ['RXD',  Reg(16, [ BF("DATA", 15, 0)], flags = Reg.READONLY | Reg.VOLATILE)]
}

To turn this into a register decoder circuit, we create a factory function, returning a `worker` process:

In [5]:
from myirl.kernel import sensitivity
from myirl.kernel.sig import ConstSig
from myirl import simulation as sim


def gen_regdec(regmap, port, clk, reset, wr, addr, idata, odata, REPORT_ILLEGAL_ACCESS = False):
    
    @sensitivity.process(clk, EDGE=clk.POS, RESET=reset)
    def worker(logic):
        """Creates a case/when flow the procedural way"""
        cw, cr = [ logic.Case(addr) for _ in range(2) ]
        N = addr.size()
                
        _reset = []
        for k, rdesc in regmap.items():
            name, rd = rdesc[0], rdesc[1]
            if rd.has_select(Reg.READONLY):
                maybe_write_sel = port[name].get_children()['select'].sel_w.set(True)
            else:
                maybe_write_sel = None
            if rd.has_select(Reg.WRITEONLY):
                maybe_read_sel = port[name].get_children()['select'].sel_r.set(True)
            else:
                maybe_read_sel = None
                                
            reg = port[name]    
            _reset += [ reg.select_reset(), ]
            s = ConstSig(k, N)
            if not rd.flags & Reg.READONLY:
                cw = cw.When(s)(reg.set(idata), maybe_write_sel)
            if not rd.flags & Reg.WRITEONLY:
                cr = cr.When(s)(reg.assign(odata), maybe_read_sel)
                
        if REPORT_ILLEGAL_ACCESS:
            cw = cw.Other(sim.raise_(ValueError("Illegal WRITE address")))
            cr = cr.Other(sim.raise_(ValueError("Illegal READ address")))
        else:
            cw = cw.Other(None)
            cr = cr.Other(None)

        
        _if = logic.If(wr == True).Then(cw).Else(cr)
 
        logic += _reset
        logic += [_if ]
    return worker


The register decoder for this specific memory mapped register has a dynamic `registerbank` dictionary passed to the interface, containing the register in/out wires. This variable argument construct is inferred to a HDL description.

## Register map decoder

The actual register map decoder consists of the code below.

In [6]:
from myirl.emulation.myhdl import *
from myirl.library.portion import *

Bool = SigType(bool)
Addr = SigType(intbv, 12)
Data = SigType(intbv, 16)

@block
def mmr_decode(
    clk : ClkSignal,
    reset : ResetSignal,
    addr : Addr,
    wr   : Bool,
    data_in : Data,
    data_out : Data.Output,
    REGDESC,
    **registerbank
):
    # We use a partially assigneable signal:
    
    idata = PASignal(intbv()[len(data_out):])
    
    # Then generate the decoder from the register map description passed:
    wk = gen_regdec(REGDESC, registerbank, clk, reset, wr, addr, data_in, idata)

    @always(clk.posedge)
    def drive():
        data_out.next = idata
            
    return instances()

**Note**: This `mmr_decode` instance can be used only once. FIXME: Signature.

## Test bench

We define an interface generation function that creates a signal dictionary out of the register description:

In [7]:
# Interface generation:

def gen_interface(rd):
    d = {}
    for k, rdesc in rd.items():
        n, reg = rdesc[0], rdesc[1]
        sig = RegisterSignal(n, reg)
        sig.rename(n)
        d[n] = sig
          
    return d

Then we simulate a few bus cyces in the test bench:

In [8]:
@block
def testbench(regdesc):
    din, dout = [ Data() for _ in range(2) ]
    a = Addr()
    clk = ClkSignal(name = 'clk')
    rst = ResetSignal(0, 1)
    wr = Signal(bool())

    
    mon_inv = Signal(intbv()[6:])
    mon_select = Signal(bool())

    interface = gen_interface(regdesc)
    
    wires = [
        mon_inv.wireup(interface['ctrl'].read.gna),
        mon_select.wireup(interface['ctrl'].select.sel_w),
    ]
    
    inst = mmr_decode(clk, rst, a, wr, din, dout, regdesc, **interface )
    
    @always(delay(2))
    def clkgen():
        clk.next = ~clk

        
    ctrl = interface['ctrl']
    stat = interface['stat']
    
    @instance
    def stim():
        rst.next = True
        wr.next = False
        a.next = 0x001
        yield clk.posedge

        for i in range(2):
            yield clk.posedge
            
        stat.read.ex.next = 0
        stat.read.mode.next = 4
        stat.read.im.next = 2

        a.next = 0x001

        rst.next = False

        yield 3 * (clk.posedge, )

        assert dout == 0x1004
        
        a.next = 0x002
        din.next = 0xfa
        wr.next = True
        yield clk.posedge
        wr.next = False
        yield clk.posedge

        assert ctrl.select.sel_w == True
        assert ctrl.write.gna == 0x3d
        yield clk.posedge
        assert ctrl.select.sel_w == False

        yield 2 * (clk.posedge, )
            
        raise StopSimulation
    
    return instances()

def test():

    tb = testbench(regdesc)
    f = tb.elab(targets.VHDL, elab_all = True)
    # Turn 'debug' on for simulation output
    run_ghdl(f, tb, debug = False, vcdfile = 'testbench.vcd')
test()

../../myirl/kernel/components.py:151: UserWarning: Base `dict` (Port 'regdesc') does not add to the interface.
Local signal instances will be created. Use BulkSignal classes instead.
  base.warnings.warn("""Base `dict` (Port '%s') does not add to the interface.
../../myirl/kernel/components.py:151: UserWarning: Base `dict` (Port 'REGDESC') does not add to the interface.
Local signal instances will be created. Use BulkSignal classes instead.
  base.warnings.warn("""Base `dict` (Port '%s') does not add to the interface.


Creating process 'mmr_decode/drive' with sensitivity (clk'rising,)
 Insert unit mmr_decode_s1_s1_s12_s1_s16_s16__dict 
Creating process 'testbench/clkgen' with sensitivity (<myirl.kernel.sensitivity.DeltaT object at 0x7fca687e6850>,)
Creating sequential 'testbench/stim' 
 Insert unit testbench__dict 
 Writing 'mmr_decode' to file /tmp/mmr_decode.vhdl 
Finished _elab in 0.0020 secs
 Writing 'testbench' to file /tmp/testbench.vhdl 
Finished _elab in 0.0563 secs
 Creating library file /tmp/module_defs.vhdl 


In [9]:
# ! cat -n {mmr_decode.ctx.path_prefix}mmr_decode.vhdl

In [10]:
#! cat {mmr_decode.ctx.path_prefix}testbench.vhdl

## Waveform display

In [11]:
import wavedraw
import nbwavedrom

In [12]:
TB = "testbench"

waveform = wavedraw.vcd2wave(TB+ ".vcd", TB + '.clk', None)
    
nbwavedrom.draw(waveform)